# Introduction
As part of Canadian tax filings, I am required to disclose my foreign held assets. All of my non-Canadian assets are held in US Accounts. 

I need to report the Name, the maximum value over the year, and the value at the end of the year, capital gains and finally dividends.

I can get the capital gains and the calculation of dividends easily through my statements. However, calculating the maximum value is a royal pain in the ass. This script should do two things, find the maximum value of each shareholding and find the value at the end of the year. 

Output should be: Name, Max Value, End Value.

In [16]:
# Import the necessary libraries
import pandas as pd
import numpy as np
from pandas_datareader import data
from datetime import datetime
import os

os.environ["TIINGO_API_KEY"] = "68da4cacdce89ec36864db8cb024b1c7daac1170" 
os.environ["IEX_API_KEY"] = "pk_bcc7601014a2490fa4e2fa0b330bc242"

## Set up
First we need to configure dates and set up which data source to use. Pandas as built in connections to a few through the 'pandas_datareader' library. 

In [18]:
# set tax year
taxYear = 2019

# Set the data source
# some data sources use "Close" others use "close"
dataSource = 'iex'
dataSourceCloseKeyWord = {'iex': 'close', 'morningstar': 'Close'}
closeValueKeyWord = dataSourceCloseKeyWord[dataSource]

Next we tell the script where to find the portfolio. This is stored in a csv file which contains: Account, Symbol, Name, Qty

In [19]:
filePath = '/Users/steveyun/Documents/DeathAndTaxes/EquityHoldings.csv'
assets = pd.read_csv(filePath)

outputFilePath = '/Users/steveyun/Documents/DeathAndTaxes/EquityHoldingTaxFiling2019.csv'

Finally we create a function to get the maximum and last asset values for a given equity ticker symbol. We restrict the data request to the calendar year. This is, after all, about filing income taxes.

In [20]:
# Create function to get max value and closing value based on year
def getMaxAndLastValues(ticker, year):
    # Get the max closing price value over the year
    # and the last closing price value in the year
    # ticker needs to be a string and a valid ticker equity ticker symbol
    # year needs to be an integer
    start = datetime(year, 1, 1)
    end = datetime(year, 12, 31)
    
    def getMaxLast(ticker, dataSource, start, end):
        priceHistory = data.DataReader(ticker, dataSource, start, end)
    
        # max value returns a Pandas Series object
        # the index is typicaly close, high, low, open, and volume
        # but the index can be capitalized or lower case
        # depending on the data source
        m = priceHistory.max()
        maxValue = m[closeValueKeyWord]
        lastValue = priceHistory.loc[max(priceHistory.index)][closeValueKeyWord]
        
        # return a two dim array: [maxValue, lastValue]
        return([maxValue, lastValue])
    
    if len(ticker) < 5: # stock 
        return(getMaxLast(ticker, 'iex', start, end))
    else:
        return(getMaxLast(ticker, 'morningstart', start, end))

## First step
The portfolio is stored in a seperate CSV file and needs to be imported. Will try importing it as a Pandas file. 

In [21]:
## Add two colums to the existing data frame: MaxValue, LastValue
## Use 0.0 to make sure that the datatype is a float, otherwise
## using 0 will make the values integers
assets['MaxValue'] = 0.0
assets['LastValue'] = 0.0

In [22]:
## Test a loop through the tickers

for index, asset in assets.iterrows():
#    print(row)
    ticker = asset['Symbol']
#    print(ticker)
    number = asset['Qty']
#    print(number)
    try:
        values = getMaxAndLastValues(ticker, taxYear)
    except: 
        values = [0, 0]
    
    #assets.set_value(index, 'MaxValue', number * values[0])
    #assets.set_value(index, 'LastValue', number * values[1])
    assets.at[index, 'MaxValue'] = number * values[0]
    assets.at[index, 'LastValue'] = number * values[1]
    

In [15]:
assets.to_csv(outputFilePath, index = False)